# Summary Buffer Memory

The same `CassandraChatMessageHistory` can power a more sophisticated kind of chat memory that automatically maintains a **summary** of the past interactions, so as to provide a memory that virtually extends arbitrarily far back in the past while fitting a finite amount of tokens.

In [1]:
from langchain.memory import CassandraChatMessageHistory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

In [2]:
from cqlsession import getCQLSession, getCQLKeyspace
cqlMode = 'astra_db' # 'astra_db'/'local'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

In [3]:
message_history = CassandraChatMessageHistory(
    session_id='summarized-conversation-4321',
    session=session,
    keyspace_name=keyspace,
    ttl_seconds = 3600,
)

In [4]:
message_history.clear()

Below is the logic to instantiate the LLM of choice. We choose to leave it in the notebooks for clarity.

In [5]:
from llm_choice import suggestLLMProvider

llmProvider = suggestLLMProvider()
# (Alternatively set llmProvider to 'GCP_VertexAI', 'OpenAI' ... manually if you have credentials)

if llmProvider == 'GCP_VertexAI':
    from langchain.llms import VertexAI
    llm = VertexAI()
    print('LLM from VertexAI')
elif llmProvider == 'OpenAI':
    from langchain.llms import OpenAI
    llm = OpenAI()
    print('LLM from OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM from OpenAI


## Use the memory with summary in a Chain

In [6]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=message_history,
    max_token_limit=180,
)

In [7]:
summaryConversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

With the verbose output you can track the summary. It starts empty and once few messages have been exchanged it starts to "incorporate" those messages that would have been lost in the past.

In [8]:
summaryConversation.predict(input='Tell me about William Tell, the hero')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about William Tell, the hero
AI:

> Finished chain.


" William Tell was a legendary Swiss hero who is best known for shooting an apple off his son's head with a crossbow. According to the legend, William was forced to demonstrate his skill with the crossbow by a tyrant from the Habsburg dynasty, Hermann Gessler. If William failed in his task, his son would face severe punishment. William was successful in this task and his feat has become a symbol of Swiss independence."

In [9]:
summaryConversation.predict(
    input="Any relation to the apple from Cupertino?"
)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about William Tell, the hero
AI:  William Tell was a legendary Swiss hero who is best known for shooting an apple off his son's head with a crossbow. According to the legend, William was forced to demonstrate his skill with the crossbow by a tyrant from the Habsburg dynasty, Hermann Gessler. If William failed in his task, his son would face severe punishment. William was successful in this task and his feat has become a symbol of Swiss independence.
Human: Any relation to the apple from Cupertino?
AI:

> Finished chain.


' I do not know.'

No messages have slided past the "recent window" yet -- correspondingly, the summary buffer is still empty:

In [10]:
memory.moving_summary_buffer

''

Continue the conversation - you might start to see the summary of past exchanges making its way to the final prompt for the LLM in the form of a "System" contribution to the _Current conversation_:

In [11]:
summaryConversation.predict(
    input="I heard the two apples may be in fact the very same fruit."
)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about William Tell, the hero
AI:  William Tell was a legendary Swiss hero who is best known for shooting an apple off his son's head with a crossbow. According to the legend, William was forced to demonstrate his skill with the crossbow by a tyrant from the Habsburg dynasty, Hermann Gessler. If William failed in his task, his son would face severe punishment. William was successful in this task and his feat has become a symbol of Swiss independence.
Human: Any relation to the apple from Cupertino?
AI:  I do not know.
Human: I heard the two apples may be in fact the very same fruit.
AI:

> Finished chain.


' That is an interesting theory, but I do not know if it is true.'

This is where the current summary buffer is stored

_(Note: depending on the length of the messages so far, at this point you might or might not see an empty summary. In any case, keep reading and it will doubtless start to be nonempty.)_

In [12]:
memory.moving_summary_buffer

''

The whole conversation is still in the "recent" window. Below is what will start to happen in the `ConversationSummaryBufferMemory` at the next chat message: starting from the message list, and the summary so far (if any), the LLM is asked to come up with a new summary which will overwrite the current one, stored in `memory.moving_summary_buffer`.

In [13]:
print(memory.predict_new_summary(
    memory.chat_memory.messages,
    memory.moving_summary_buffer,
))


The human asked the AI about William Tell, the hero, and the AI responded that he was a legendary Swiss hero best known for shooting an apple off his son's head with a crossbow. The human then asked if there was any relation to the apple from Cupertino and the AI replied that it did not know, but the human suggested that the two apples may be the same fruit, to which the AI responded that it was an interesting theory but it did not know if it was true.


In [14]:
summaryConversation.predict(
    input='Ok, enough with apples. Let\'s talk starfish now.'
)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about William Tell, the hero
AI:  William Tell was a legendary Swiss hero who is best known for shooting an apple off his son's head with a crossbow. According to the legend, William was forced to demonstrate his skill with the crossbow by a tyrant from the Habsburg dynasty, Hermann Gessler. If William failed in his task, his son would face severe punishment. William was successful in this task and his feat has become a symbol of Swiss independence.
Human: Any relation to the apple from Cupertino?
AI:  I do not know.
Human: I heard the two apples may be in fact the very same fruit.
AI:  That is an interesting theory, but I do not know if it is true.
Human: Ok, enough wit

" Sure! Starfish, also known as sea stars, are a type of invertebrate found in all of the world's oceans. They have a wide variety of shapes and colors, and can range in size from a few millimeters to over a meter in diameter. They feed on clams, mussels, and other invertebrates, using their suction-cup like feet to pry food from the ocean floor."

In [15]:
memory.moving_summary_buffer

"\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human then inquires about William Tell, the hero. The AI explains that William Tell was a legendary Swiss hero who is best known for shooting an apple off his son's head with a crossbow. According to the legend, William was forced to demonstrate his skill with the crossbow by a tyrant from the Habsburg dynasty, Hermann Gessler. If William failed in his task, his son would face severe punishment. William was successful in this task and his feat has become a symbol of Swiss independence."

Now the chat history has presumably reached a certain threshold and there is a summary. In the verbose output below, you will see it appearing in the prompt as a "System" role in the conversation recap.

In [16]:
summaryConversation.predict(
    input='I hear that some of those destroy marine ecosystems.'
)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human then inquires about William Tell, the hero. The AI explains that William Tell was a legendary Swiss hero who is best known for shooting an apple off his son's head with a crossbow. According to the legend, William was forced to demonstrate his skill with the crossbow by a tyrant from the Habsburg dynasty, Hermann Gessler. If William failed in his task, his son would face severe punishment. William was successful in this task and his feat has become a symbol of Swiss independence.
Human: Tell me

'  Yes, some species of starfish can be invasive and can cause significant damage to native marine ecosystems. In certain parts of the world, the populations of certain species of starfish have grown unchecked, resulting in the depletion of other species and the disruption of the food chain.'

In [17]:
memory.moving_summary_buffer

"\n\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human then inquires about William Tell, the hero. The AI explains that William Tell was a legendary Swiss hero who is best known for shooting an apple off his son's head with a crossbow. According to the legend, William was forced to demonstrate his skill with the crossbow by a tyrant from the Habsburg dynasty, Hermann Gessler. If William failed in his task, his son would face severe punishment. William was successful in this task and his feat has become a symbol of Swiss independence. The human then asks if there is any relation to the apple from Cupertino and the AI does not know, but the human suggests the two apples may be the same fruit, to which the AI says it is an interesting theory, but they do not know if it is true."

From this point on, the summary will not grow indefinitely in size, but will rather try to cover a longer and longer conversation in a fixed amount of text.

In [18]:
print(memory.predict_new_summary(
    memory.chat_memory.messages,
    memory.moving_summary_buffer,
))


The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human then inquires about William Tell, the hero and the AI explains that William Tell was a legendary Swiss hero who is best known for shooting an apple off his son's head with a crossbow. According to the legend, William was forced to demonstrate his skill with the crossbow by a tyrant from the Habsburg dynasty, Hermann Gessler. If William failed in his task, his son would face severe punishment. William was successful in this task and his feat has become a symbol of Swiss independence. When asked if there is any relation to the apple from Cupertino, the AI does not know, but the human suggests the two apples may be the same fruit, to which the AI says it is an interesting theory, but they do not know if it is true. The conversation then moves on to starfish and the AI explains that starfish, also known as